## Homework: Deep Jazz

In [1]:
import numpy as np
from music21 import stream, midi, tempo, note

from grammar import unparse_grammar
from preprocess import get_musical_data, get_corpus_data

from qa import prune_grammar, prune_notes, clean_up_notes
from generator import __sample, __generate_grammar, __predict

In [2]:
N_epochs = 128  # default
data_fn = 'midi/' + 'original_metheny.mid'  # 'And Then I Knew' by Pat Metheny
out_fn = 'midi/' 'deepjazz_on_metheny...' + str(N_epochs)

In [3]:
max_len = 20
max_tries = 1000
diversity = 0.5

# musical settings
bpm = 130

# get data
chords, abstract_grammars = get_musical_data(data_fn)
corpus, values, val_indices, indices_val = get_corpus_data(abstract_grammars)
print('corpus length:', len(corpus))
print('total # of values:', len(values))

corpus length: 193
total # of values: 78


In [4]:
chords[0]

[<music21.instrument.Piano Piano>,
 <music21.tempo.MetronomeMark Quarter=112.0>,
 <music21.key.Key of G major>,
 <music21.meter.TimeSignature 4/4>]

## Task

Replace previous model with equivalent in prettytensor or tf.slim

Try to make you code as compact as possible

In [5]:
def get_pretty_tensor_model(max_len, N_values):
    import tflearn
    tflearn.init_graph(gpu_memory_fraction=0.33)
    
    net = tflearn.input_data(shape=[None, max_len, N_values])
    net = tflearn.lstm(net, 128, dropout=(1.0, 0.8), return_seq=True)
    net = tflearn.lstm(net, 128, dropout=(1.0, 0.8), return_seq=False)
    net = tflearn.fully_connected(net, N_values, activation='softmax')
    net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    model.rpredict = model.predict
    model.predict = lambda x, **kwargs: model.rpredict(x)
    return model

In [6]:
get_model = get_pretty_tensor_model

In [7]:
import numpy as np

''' Build a 2-layer LSTM from a training corpus '''


def build_model(corpus, val_indices, max_len, N_epochs=128):
    # number of different values or words in corpus
    N_values = len(set(corpus))

    # cut the corpus into semi-redundant sequences of max_len values
    step = 3
    sentences = []
    next_values = []
    for i in range(0, len(corpus) - max_len, step):
        sentences.append(corpus[i: i + max_len])
        next_values.append(corpus[i + max_len])
    print('nb sequences:', len(sentences))

    # transform data into binary matrices
    X = np.zeros((len(sentences), max_len, N_values), dtype=np.bool)
    y = np.zeros((len(sentences), N_values), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, val in enumerate(sentence):
            X[i, t, val_indices[val]] = 1
        y[i, val_indices[next_values[i]]] = 1

    model = get_model(max_len, N_values)
    print(max_len)
    print(N_values)
    print(X.shape)
    model.fit(X, y, batch_size=128, n_epoch=N_epochs)

    return model


In [8]:
# build model
model = build_model(corpus=corpus, val_indices=val_indices,
                         max_len=max_len, N_epochs=N_epochs)

# set up audio stream
out_stream = stream.Stream()

# generation loop
curr_offset = 0.0
loopEnd = len(chords)
for loopIndex in range(1, loopEnd):
    # get chords from file
    curr_chords = stream.Voice()
    for j in chords[loopIndex]:
        curr_chords.insert((j.offset % 4), j)

    # generate grammar
    curr_grammar = __generate_grammar(model=model, corpus=corpus,
                                      abstract_grammars=abstract_grammars,
                                      values=values, val_indices=val_indices,
                                      indices_val=indices_val,
                                      max_len=max_len, max_tries=max_tries,
                                      diversity=diversity)

    curr_grammar = curr_grammar.replace(' A', ' C').replace(' X', ' C')

    # Pruning #1: smoothing measure
    curr_grammar = prune_grammar(curr_grammar)

    # Get notes from grammar and chords
    curr_notes = unparse_grammar(curr_grammar, curr_chords)

    # Pruning #2: removing repeated and too close together notes
    curr_notes = prune_notes(curr_notes)

    # quality assurance: clean up notes
    curr_notes = clean_up_notes(curr_notes)

    # print # of notes in curr_notes
    print('After pruning: %s notes' % (len([i for i in curr_notes
                                            if isinstance(i, note.Note)])))

    # insert into the output stream
    for m in curr_notes:
        out_stream.insert(curr_offset + m.offset, m)
    for mc in curr_chords:
        out_stream.insert(curr_offset + mc.offset, mc)

    curr_offset += 4.0

out_stream.insert(0.0, tempo.MetronomeMark(number=bpm))

# Play the final stream through output (see 'play' lambda function above)
#play = lambda x: midi.realtime.StreamPlayer(x).play()
#play(out_stream)

# save stream
mf = midi.translate.streamToMidiFile(out_stream)
mf.open(out_fn, 'wb')
mf.write()
mf.close()    

nb sequences: 58


/opt/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
20
78
(58, 20, 78)
---------------------------------
Run id: A1A355
Log directory: /tmp/tflearn_logs/
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'list' object has no attribute 'name'
---------------------------------
Training samples: 58
Validation samples: 0
--
Training Step: 1  | time: 0.633s
| Adam | epoch: 001 | loss: 0.00000 -- iter: 58/58
--
Training Step: 2  | total loss: 3.92111 | time: 0.072s
| Adam | epoch: 002 | loss: 3.92111 -- iter: 58/58
--
Training Step: 3  | total loss: 4.27565 | time: 0.070s
| Adam | epoch: 003 | loss: 4.27565 -- iter: 58/58
--
Training Step: 4  | total loss: 4.33303 | time: 0.071s
| Adam | epoch: 004 | loss: 4.33303 -- iter: 58/58
--
Training Step: 5  | total loss: 4.34293 | time: 0.069s
| Adam | epoch: 005 | loss: 4.34293 -- iter: 58/58
--
Training Step: 6  | total loss: 4.34384 | time: 0.069s
| Adam | epoch: 006 |

Training Step: 63  | total loss: 3.46194 | time: 0.082s
| Adam | epoch: 063 | loss: 3.46194 -- iter: 58/58
--
Training Step: 64  | total loss: 3.46386 | time: 0.070s
| Adam | epoch: 064 | loss: 3.46386 -- iter: 58/58
--
Training Step: 65  | total loss: 3.46435 | time: 0.070s
| Adam | epoch: 065 | loss: 3.46435 -- iter: 58/58
--
Training Step: 66  | total loss: 3.46464 | time: 0.072s
| Adam | epoch: 066 | loss: 3.46464 -- iter: 58/58
--
Training Step: 67  | total loss: 3.46630 | time: 0.071s
| Adam | epoch: 067 | loss: 3.46630 -- iter: 58/58
--
Training Step: 68  | total loss: 3.46812 | time: 0.075s
| Adam | epoch: 068 | loss: 3.46812 -- iter: 58/58
--
Training Step: 69  | total loss: 3.46815 | time: 0.071s
| Adam | epoch: 069 | loss: 3.46815 -- iter: 58/58
--
Training Step: 70  | total loss: 3.46892 | time: 0.072s
| Adam | epoch: 070 | loss: 3.46892 -- iter: 58/58
--
Training Step: 71  | total loss: 3.46916 | time: 0.071s
| Adam | epoch: 071 | loss: 3.46916 -- iter: 58/58
--
Training S

Training Step: 128  | total loss: 3.46294 | time: 0.069s
| Adam | epoch: 128 | loss: 3.46294 -- iter: 58/58
--
After pruning: 15 notes
After pruning: 12 notes
After pruning: 13 notes
After pruning: 15 notes
After pruning: 14 notes
After pruning: 12 notes
After pruning: 13 notes
After pruning: 14 notes
After pruning: 14 notes
After pruning: 16 notes
After pruning: 13 notes
After pruning: 15 notes
After pruning: 14 notes
After pruning: 14 notes
After pruning: 13 notes
After pruning: 14 notes
After pruning: 15 notes
After pruning: 9 notes


You can play generated sample using any midi player

Under linux I prefer timidity

In [9]:
!! timidity midi/deepjazz_on_metheny...128_epochs.midi

['/bin/bash: timidity: command not found']